In [1]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "Bookshop"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=Bookshop;Trusted_Connection=yes 

Successfully connected to Bookshop!


In [7]:
from sqlalchemy import text

search = input("Please enter a book title")

print(f"Book Search : {search}\n")

query = f"""select top 5
	b.Title,
	i.Amount,
	s.Name as 'Shop'

from
	books b
	join Inventory i on b.ISBN13 = i.ISBN13
	join Shops s on i.Shop_Id = s.Shop_Id
where 
    b.Title like '%{search}%'"""
    
with engine.connect() as conn:    
    result = conn.execute(text(query))

    print(f"{'Book Title'.ljust(8)}{'Amount'.ljust(50)}{'Shop'}")
    
    for book in result:
        print(f"{str(book.Title).ljust(8)}{str(book.Amount).ljust(50)}{book.Shop}")

    print(query)

Book Search : The Colour Of Magic

Book TitleAmount                                            Shop
The Colour Of Magic34                                                Bookworm
The Colour Of Magic76                                                Cosy Reads
The Colour Of Magic87                                                Book Knook
select top 5
	b.Title,
	i.Amount,
	s.Name as 'Shop'

from
	books b
	join Inventory i on b.ISBN13 = i.ISBN13
	join Shops s on i.Shop_Id = s.Shop_Id
where 
    b.Title like '%The Colour Of Magic%'
